In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install wandb

In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [5]:
df_train = pd.read_csv('/content/drive/MyDrive/Redes Neurais/Github/Redes-Neurais/Titanic/train.csv')

#Simplista tirar o NAN

df_train = df_train.dropna()

### Tratar a base

In [6]:
df_train.head(2)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


In [7]:
y = df_train[['Transported']]
y['Transported'] = y['Transported'].apply(lambda x: 1 if x == True else 0)

train = df_train.drop(columns = ['PassengerId', 'Name', 'Transported'])

## Dummie dos categóricos (testar sem dummies fazendo os encoders dentro da coluna)

train = pd.concat((train, pd.get_dummies(train[['HomePlanet', 'CryoSleep', 'Destination', 'VIP']])), axis=1)
train = train.drop(['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
## Parte de tratamento das cabines. Pode ignorar

train = train.drop(train['Cabin'][train['Cabin'] == 'NoneCabin'].index, axis=0)
train['Cabin'] = train['Cabin'].fillna(train['Cabin'].mode()[0])

first, sec, thrd = [], [], []
mem = []
for i in range(train['Cabin'].shape[0]):
    mem = train['Cabin'].iloc[i].split('/')
    first.append(mem[0])
    sec.append(mem[1])
    thrd.append(mem[2])
first = pd.Series(first)
first.index = train['Cabin'].index
sec = pd.Series(sec)
sec.index = train['Cabin'].index
thrd = pd.Series(thrd) 
thrd.index = train['Cabin'].index
train['CabinClass'] = first
train['CabinNumber'] = sec
train['CabinSeat'] = thrd

'''first, sec, thrd = [], [], []
mem = []
for i in range(test['Cabin'].shape[0]):
    mem = test['Cabin'].iloc[i].split('/')
    first.append(mem[0])
    sec.append(mem[1])
    thrd.append(mem[2])
first = pd.Series(first)
first.index = test['Cabin'].index
sec = pd.Series(sec)
sec.index = test['Cabin'].index
thrd = pd.Series(thrd) 
thrd.index = test['Cabin'].index
test['CabinClass'] = first
test['CabinNumber'] = sec
test['CabinSeat'] = thrd'''

train = train.drop('Cabin',axis=1)
'''test = test.drop('Cabin',axis=1)

test.dtypes[(test.dtypes == "object") == True].index.tolist()

test.dtypes[(test.dtypes != "object") == True].index.tolist() == train.dtypes[(train.dtypes != "object") == True].index.tolist()
'''
train = pd.concat((train, pd.get_dummies(train['CabinClass'])), axis=1)
train = pd.concat((train, pd.get_dummies(train['CabinSeat'])), axis=1)
train = train.drop(['CabinClass','CabinSeat'],axis=1)

In [9]:
### Normalizar

transformer = Normalizer().fit(train) 
transformer
train=transformer.transform(train)


'''#Splitar em treinamento e teste

x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.3)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)'''

x_train = train
y_train = y

###Estrutura da rede

In [24]:
wandb.init(settings=wandb.Settings(start_method="thread"),project="WandB tutorial")

accuracy,▁▇▇▇████████████████████████████████████
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██▁▁▂▂▃▃▄▄▁▁▂▂▃▃▄▄▁▁▂▂▃▃▄▅
loss,█▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▃▅▇▇▇█▇▇▇██████████████▇▇▇████████▇██▇
val_loss,█▆▆▅▄▄▄▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▃▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▂
accuracy,0.79864
best_epoch,8
best_val_loss,0.43592
epoch,9
loss,0.42885
val_accuracy,0.80333


In [26]:
model = Sequential() 
model.add(Dense(192, activation='relu', input_dim=train.shape[1]))
model.add(Dense(192, activation='relu', input_dim=train.shape[1]))
model.add(Dense(192, activation='relu', input_dim=train.shape[1]))
model.add(Dense(192, activation='relu', input_dim=train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) 

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy']) 

In [23]:
model.fit(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[WandbCallback()])

Epoch 1/10
166/166 [==============================] - 1s 5ms/step - loss: 0.4370 - accuracy: 0.7950 - val_loss: 0.4489 - val_accuracy: 0.8041
Epoch 2/10
166/166 [==============================] - 1s 3ms/step - loss: 0.4365 - accuracy: 0.7983 - val_loss: 0.4484 - val_accuracy: 0.8041
Epoch 3/10
151/166 [==========================>...] - ETA: 0s - loss: 0.4343 - accuracy: 0.7955

wandb: Adding directory to artifact (/content/wandb/run-20221130_010425-3j2ljqp6/files/model-best)... Done. 0.1s


166/166 [==============================] - 1s 9ms/step - loss: 0.4345 - accuracy: 0.7945 - val_loss: 0.4448 - val_accuracy: 0.8056
Epoch 4/10
166/166 [==============================] - 1s 3ms/step - loss: 0.4336 - accuracy: 0.7979 - val_loss: 0.4448 - val_accuracy: 0.8048
Epoch 5/10
156/166 [===========================>..] - ETA: 0s - loss: 0.4361 - accuracy: 0.7955

wandb: Adding directory to artifact (/content/wandb/run-20221130_010425-3j2ljqp6/files/model-best)... Done. 0.1s


166/166 [==============================] - 2s 12ms/step - loss: 0.4336 - accuracy: 0.7966 - val_loss: 0.4412 - val_accuracy: 0.8041
Epoch 6/10
166/166 [==============================] - 1s 5ms/step - loss: 0.4328 - accuracy: 0.7983 - val_loss: 0.4431 - val_accuracy: 0.8018
Epoch 7/10
166/166 [==============================] - 0s 3ms/step - loss: 0.4304 - accuracy: 0.7967 - val_loss: 0.4479 - val_accuracy: 0.8041
Epoch 8/10
166/166 [==============================] - 0s 3ms/step - loss: 0.4286 - accuracy: 0.7949 - val_loss: 0.4421 - val_accuracy: 0.8041
Epoch 9/10
154/166 [==========================>...] - ETA: 0s - loss: 0.4324 - accuracy: 0.7967

wandb: Adding directory to artifact (/content/wandb/run-20221130_010425-3j2ljqp6/files/model-best)... Done. 0.1s


166/166 [==============================] - 1s 9ms/step - loss: 0.4309 - accuracy: 0.7981 - val_loss: 0.4359 - val_accuracy: 0.8056
Epoch 10/10
166/166 [==============================] - 0s 3ms/step - loss: 0.4288 - accuracy: 0.7986 - val_loss: 0.4504 - val_accuracy: 0.8033


In [ ]:
##

####Tunar hiperparametro

In [20]:
sweep_config = {
  'method': 'random', 
  'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [32, 64, 128, 256]
      },
      'learning_rate':{
          'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
      }
  }
}

In [29]:
def sweep_train(config_defaults=None):
    # Set default values
    config_defaults = {
        "batch_size": 64,
        "learning_rate": 0.01
    }
    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)  # this gets over-written in the Sweep

    # Specify the other hyperparameters to the configuration, if any
    '''    wandb.config.epochs = 2
    wandb.config.log_step = 20
    wandb.config.val_log_step = 50
    wandb.config.architecture_name = "MLP"
    wandb.config.dataset_name = "MNIST"'''

    


    model.fit(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[WandbCallback()])

In [22]:
sweep_id = wandb.sweep(sweep_config, project="sweeps-tensorflow")

Create sweep with ID: wqux4o69
Sweep URL: https://wandb.ai/lucasalmada/sweeps-tensorflow/sweeps/wqux4o69


In [23]:
wandb.agent(sweep_id, function=sweep_train, count=10)

wandb: Agent Starting Run: hi28fnb8 with config:
wandb: 	batch_size: 32
wandb: 	learning_rate: 0.0005
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/10
166/166 [==============================] - ETA: 0s - loss: 0.4079 - accuracy: 0.8024

wandb: Adding directory to artifact (/content/wandb/run-20221130_015056-hi28fnb8/files/model-best)... Done. 0.1s


166/166 [==============================] - 2s 11ms/step - loss: 0.4079 - accuracy: 0.8024 - val_loss: 0.4622 - val_accuracy: 0.8033
Epoch 2/10
156/166 [===========================>..] - ETA: 0s - loss: 0.4118 - accuracy: 0.8007

wandb: Adding directory to artifact (/content/wandb/run-20221130_015056-hi28fnb8/files/model-best)... Done. 0.1s


...

wandb: Adding directory to artifact (/content/wandb/run-20221130_015454-a0g06jre/files/model-best)... Done. 0.1s


166/166 [==============================] - 2s 9ms/step - loss: 0.3969 - accuracy: 0.8109 - val_loss: 0.4204 - val_accuracy: 0.8048
Epoch 7/10
160/166 [===========================>..] - ETA: 0s - loss: 0.3952 - accuracy: 0.8082

wandb: Adding directory to artifact (/content/wandb/run-20221130_015454-a0g06jre/files/model-best)... Done. 0.1s


166/166 [==============================] - 2s 9ms/step - loss: 0.3960 - accuracy: 0.8083 - val_loss: 0.4045 - val_accuracy: 0.8064
Epoch 8/10
166/166 [==============================] - 1s 4ms/step - loss: 0.3933 - accuracy: 0.8075 - val_loss: 0.4344 - val_accuracy: 0.8056
Epoch 9/10
166/166 [==============================] - 1s 3ms/step - loss: 0.3975 - accuracy: 0.8073 - val_loss: 0.4621 - val_accuracy: 0.8116
Epoch 10/10
166/166 [==============================] - 1s 3ms/step - loss: 0.3935 - accuracy: 0.8055 - val_loss: 0.4887 - val_accuracy: 0.8101


accuracy,▄▄▃█▆▅▃▃▂▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,▃▂▂▃▄▇▅▁█▁
val_accuracy,▁▇▂▆▄▂▄▃█▇
val_loss,▄▄▅▃▃▂▁▃▆█
accuracy,0.80545
best_epoch,6
best_val_loss,0.4045
epoch,9
loss,0.39352
val_accuracy,0.81014


In [16]:
wandb agent lucasalmada/sweeps-tensorflow/xuwx7hgd


SyntaxError: ignored

In [31]:
wandb.agent('ig4a3led', function=sweep_train, count=10)

wandb: Agent Starting Run: ydphoee3 with config:
wandb: 	architecture_name: MLP
wandb: 	batch_size: 38
wandb: 	dataset_name: MNIST
wandb: 	epochs: 3
wandb: 	learning_rate: 0.018443613969138465
wandb: 	log_step: 26
wandb: 	val_log_step: 99
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/50
163/166 [============================>.] - ETA: 0s - loss: 0.4056 - accuracy: 0.8014

wandb: Adding directory to artifact (/content/wandb/run-20221130_022204-ydphoee3/files/model-best)... Done. 0.1s


166/166 [==============================] - 3s 16ms/step - loss: 0.4058 - accuracy: 0.8019 - val_loss: 0.4872 - val_accuracy: 0.8064
Epoch 2/50
165/166 [============================>.] - ETA: 0s - loss: 0.4101 - accuracy: 0.8068

wandb: Adding directory to artifact (/content/wandb/run-20221130_022204-ydphoee3/files/model-best)... Done. 0.1s


...

wandb: Adding directory to artifact (/content/wandb/run-20221130_023327-cajla11c/files/model-best)... Done. 0.1s


166/166 [==============================] - 2s 12ms/step - loss: 0.1840 - accuracy: 0.9129 - val_loss: 1.1585 - val_accuracy: 0.7640
Epoch 7/50
158/166 [===========================>..] - ETA: 0s - loss: 0.1651 - accuracy: 0.9159

wandb: Adding directory to artifact (/content/wandb/run-20221130_023327-cajla11c/files/model-best)... Done. 0.1s


166/166 [==============================] - 3s 16ms/step - loss: 0.1686 - accuracy: 0.9152 - val_loss: 1.0798 - val_accuracy: 0.7572
Epoch 8/50
166/166 [==============================] - 1s 7ms/step - loss: 0.1564 - accuracy: 0.9228 - val_loss: 1.0969 - val_accuracy: 0.7579
Epoch 9/50
166/166 [==============================] - 1s 5ms/step - loss: 0.1536 - accuracy: 0.9234 - val_loss: 1.2471 - val_accuracy: 0.7519
Epoch 10/50
166/166 [==============================] - 1s 5ms/step - loss: 0.1493 - accuracy: 0.9232 - val_loss: 1.1550 - val_accuracy: 0.7617
Epoch 11/50
166/166 [==============================] - 1s 7ms/step - loss: 0.1611 - accuracy: 0.9190 - val_loss: 1.2671 - val_accuracy: 0.7436
Epoch 12/50
166/166 [==============================] - 1s 5ms/step - loss: 0.1574 - accuracy: 0.9235 - val_loss: 1.1557 - val_accuracy: 0.7572
Epoch 13/50
166/166 [==============================] - 1s 6ms/step - loss: 0.1482 - accuracy: 0.9264 - val_loss: 1.2145 - val_accuracy: 0.7761
Epoch 14/50


accuracy,▅▅▅▁▁▂▅▅▄▆▇▆▄▄▄▄▅▆▆▅▄▃▆▇▆▅▇█▅▄▅▇▃▅▇██▇▇▅
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▄▄▄▇█▆▄▄▅▄▃▂▅▅▆▆▄▃▃▃▅▇▄▂▃▃▂▂▄▅▄█▇▄▂▁▂▂▃▅
val_accuracy,▆▇▇▆▆▅▅▄▃▅██▆▆▁▅▇▇▅▆▇▅▆▆▆▅▇▅▆▆▄▅▃▅▅▅▄▃▄▄
val_loss,▃▃▃▃▂▁▁▄▄▂▃▆▄▅▇▃▃▃▅▇▃▂▄▄▄▆▆█▆▇▅█▃▄▄▆█▇▅▅
accuracy,0.92316
best_epoch,6
best_val_loss,1.07981
epoch,49
loss,0.16343
val_accuracy,0.75113


In [24]:
sweep_id

'wqux4o69'